In [1]:
import torch
import data_utils
from models.DeepAnT import DeepAnT_CNN, DeepAnT_LSTM
from training.DeepAnT_train import train_kdd99, train_financial
import matplotlib.pyplot as plt

In [2]:
# Try to use GPU if available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


# Hyperparameters

In [3]:
# choose between CNN and LSTM based DeepAnT
model_type = "CNN"
# choose dataset (e.g., kdd99, aapl, gm, axp, etc.)
dataset = "kdd99"

In [4]:
if model_type == "LSTM":
    # Data
    num_features = 7
    seq_length = 30
    seq_stride = 10
    gen_seq_len = seq_length
    batch_size = 8
    # Model
    hidden_dim = 128
    layers = 4
    anm_det_thr = 0.5
    # Training
    num_epochs = 100
    lr=1e-5
    wd=5e-6

In [5]:
if model_type == "CNN":
    # Data
    num_features = 34
    seq_length = 30
    seq_stride = 1
    gen_seq_len = 1
    batch_size = 256
    # Model
    dense_dim = 448 
    num_channels = 64
    kernel_size = 3
    anm_det_thr = 0.5
    # Training
    num_epochs = 100
    lr=1e-5
    wd=5e-6

# Dataset

In [6]:
if dataset == "kdd99":
    train_dl, test_dl = data_utils.kdd99(seq_length, seq_stride, num_features, gen_seq_len, batch_size, deepant=True)
else:
    file_path = 'data/financial_data/Stocks/' + dataset + '.us.txt'
    tscv_dl_list = data_utils.load_stock_as_crossvalidated_timeseries(file_path, seq_length, seq_stride, gen_seq_len, batch_size, normalise=True)

# Model 

In [7]:
if model_type == "LSTM":
    model = DeepAnT_LSTM(num_features,hidden_dim,layers,anm_det_thr).to(DEVICE)
elif model_type == "CNN":
    model = DeepAnT_CNN(seq_length,num_features, kernel_size, dense_dim, num_channels, anm_det_thr).to(DEVICE)

# Loss & Optimizer

In [8]:
loss_function = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(list(model.parameters()), lr=lr, weight_decay=wd)

# Training & Evaluation

In [9]:
if dataset == "kdd99":
    train_kdd99(train_dl, test_dl, model, optimizer, loss_function, num_epochs, DEVICE)
else:
    train_financial(tscv_dl_list, model, optimizer, loss_function, num_epochs, DEVICE)

Epoch 0 loss: 0.4478282312072837
Epoch 1 loss: 0.21379268170759577
Epoch 2 loss: 0.13833458478145833
Epoch 3 loss: 0.10143014830170462
Epoch 4 loss: 0.08083137006083022
Epoch 5 loss: 0.0676886619636293
Epoch 6 loss: 0.05809296394156145
Epoch 7 loss: 0.051758765446892854
Epoch 8 loss: 0.0474901528215959
Epoch 9 loss: 0.04423126701857347
Epoch 10 loss: 0.041280782243604
Epoch 11 loss: 0.03885998937539149
Epoch 12 loss: 0.03732803029474083
Epoch 13 loss: 0.03604963183204627
Epoch 14 loss: 0.035038715726104605
Epoch 15 loss: 0.034142096125481344
Epoch 16 loss: 0.03326069459091901
Epoch 17 loss: 0.03255846147952388
Epoch 18 loss: 0.03152491681872917
Epoch 19 loss: 0.03056418220014479
Epoch 20 loss: 0.02964132027526742
Epoch 21 loss: 0.029046731581763078
Epoch 22 loss: 0.02856165246252084
Epoch 23 loss: 0.028095763378459104
Epoch 24 loss: 0.02773205123241511
Epoch 25 loss: 0.027316071405925423
Epoch 26 loss: 0.027032986312975762
Epoch 27 loss: 0.02665660085142147
Epoch 28 loss: 0.02630948032

In [10]:
from training.DeepAnT_train import evaluate
evaluate(model, test_dl, 1, DEVICE)

Prediction time: 2.998967409133911
Acc: 0.8029934423575129, Pre: 0.8029934423575129, Rec: 0.8435233160621761
EM: 0.00015824399885063417, MV: 1.1945177592209758e-14


# Generated Samples

In [11]:
batch = next(iter(tscv_dl_list[4][1]))
x = batch[0][0] if (model_type == "LSTM") else batch[0]
y = batch[1][0] if (model_type == "LSTM") else batch[1][:seq_length].squeeze()
z = model(x.to(DEVICE)).cpu().detach() 
if (model_type == "CNN"): z = z[:seq_length]

NameError: name 'tscv_dl_list' is not defined

In [ ]:
# Generated Sample
plt.plot(z)

In [ ]:
# Real Sample
plt.plot(y)